In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
with open('C:\\Users\\Harry\\Documents\\DS\\Datasets\\senti_set\\dataset\\sentences_list.pkl', 'rb') as f:
    sentences = pickle.load(f)
    
with open('C:\\Users\\Harry\\Documents\\DS\\Datasets\\senti_set\\dataset\\categories_list.pkl', 'rb') as f:
    categories = pickle.load(f)

### Preprocessing

In [4]:
def lowercase(sentences):
    for i in range(len(sentences)):
        sentences[i] = sentences[i].lower()

In [5]:
import re
def remove_numbers(sentences):
    for i in range(len(sentences)):
        sentences[i] = re.sub(r'\d+', ' NUMBER ', sentences[i])

In [6]:
import string
translator = str.maketrans('', '', string.punctuation)
def remove_punctuations(sentences):
    for i in range(len(sentences)):
        sentences[i] = sentences[i].translate(translator)

In [7]:
def remove_whitespaces(sentences):
    for i in range(len(sentences)):
        sentences[i] = sentences[i].strip()

In [8]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sw = stopwords.words("english")
def remove_stopwords_tokenize(sentences):
    for i in range(len(sentences)):
        tokens = word_tokenize(sentences[i])
        sentences[i] = [j for j in tokens if not j in sw]

In [9]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(sentences):
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            sentences[i][j] = lemmatizer.lemmatize(sentences[i][j])

In [10]:
def preprocess(sentences):
    lowercase(sentences)
    remove_punctuations(sentences)
    remove_numbers(sentences)
    remove_whitespaces(sentences)
    remove_stopwords_tokenize(sentences)
    lemmatize(sentences)

In [11]:
preprocess(sentences)

In [12]:
sentences[2]

['went', 'gym', 'morning', 'yoga']

In [13]:
sentences_copy = []
for i in sentences:
    sentences_copy.append(' '.join(i))
sentences_copy[0]

'went successful date someone felt sympathy connection'

In [14]:
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(sentences_copy)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=0,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [15]:
a = vectorizer.vocabulary_
b = []
for i in a.keys():
    b.append((a[i], i))
b.sort()

In [16]:
count = 0
for i in a.keys():
  count += 1

count

18724

In [17]:
import numpy as np
MAX_LEN = 15
sent_vectors = np.zeros((83643, MAX_LEN))

for id, sent in enumerate(sentences):
  for i, word in enumerate(sent):
    try:
      sent_vectors[id][i] = a[word]
    except:
      pass

In [18]:
sent_vectors[0]

array([18220., 16094.,  4080., 15400.,  5945., 16343.,  3432.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.])

In [0]:
# with open('/content/drive/My Drive/NLP - UT/vect_sentences.pkl', 'wb') as f:
#   pickle.dump(sent_vectors, f, protocol=4)

In [19]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [0]:
from keras import models
from keras.layers import Dense, Dropout, Embedding, LSTM, TimeDistributed, SpatialDropout1D
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.utils.vis_utils import model_to_dot

# model = models.Sequential()
# model.add(Dense(512, activation='relu', input_shape=(30,)))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(7, activation='softmax'))
use_dropout = True
model = models.Sequential()
model.add(Embedding(18724, 300, input_length=MAX_LEN))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(150, dropout=0.7, recurrent_dropout=0.7, return_sequences=True))
model.add(LSTM(75, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(7, activation='softmax'))

In [0]:
from keras import models
from keras.layers import Dense, Dropout, Embedding, LSTM, TimeDistributed, SpatialDropout1D, Bidirectional
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.utils.vis_utils import model_to_dot

# model = models.Sequential()
# model.add(Dense(512, activation='relu', input_shape=(30,)))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(7, activation='softmax'))
use_dropout = True
model = models.Sequential()
model.add(Embedding(18724, 300, input_length=MAX_LEN))
model.add(SpatialDropout1D(0.7))
model.add(Bidirectional(LSTM(150, dropout=0.7, recurrent_dropout=0.7)))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [0]:
NUM_CLASSES = 7
x_train = sent_vectors[:67644, :]
x_test = sent_vectors[67644:, :]
categories = np.asarray(categories)
y_train = categories[:67644]
y_test = categories[67644:]

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [25]:
y_train[2]

array([0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [35]:
BATCH_SIZE = 128
EPOCHS = 50
from keras.callbacks import ModelCheckpoint
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/NLP - UT/model_4.h5',monitor="val_acc", save_best_only=True, save_weights_only=False)
from keras.optimizers import Adam
# Compile model
model.compile(optimizer=Adam(lr=0.003),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          callbacks=[mcp],
          verbose=1,
          validation_data=(x_test, y_test))

Train on 67644 samples, validate on 15999 samples
Epoch 1/50
67644/67644 [==============================] - 47s 690us/step - loss: 0.9619 - acc: 0.6584 - val_loss: 0.7206 - val_acc: 0.7490
Epoch 2/50
67644/67644 [==============================] - 42s 623us/step - loss: 0.7180 - acc: 0.7551 - val_loss: 0.7050 - val_acc: 0.7589
Epoch 3/50
67644/67644 [==============================] - 42s 616us/step - loss: 0.6536 - acc: 0.7781 - val_loss: 0.7193 - val_acc: 0.7657
Epoch 4/50
67644/67644 [==============================] - 42s 626us/step - loss: 0.6188 - acc: 0.7907 - val_loss: 0.7348 - val_acc: 0.7657
Epoch 5/50
67644/67644 [==============================] - 42s 628us/step - loss: 0.5955 - acc: 0.7976 - val_loss: 0.7521 - val_acc: 0.7665
Epoch 6/50
67644/67644 [==============================] - 42s 627us/step - loss: 0.5826 - acc: 0.8018 - val_loss: 0.7798 - val_acc: 0.7661
Epoch 7/50
67644/67644 [==============================] - 42s 624us/step - loss: 0.5716 - acc: 0.8070 - val_loss: 0.

KeyboardInterrupt: ignored